# Downscaling all the Orthomosaics for all projects (RGB & NDVI)

In [2]:
import os
import rasterio
from rasterio.enums import Resampling

In [3]:
def find_orthomosaic_files(root_folder, search_config):
    """
    Finds orthomosaic files based on a dictionary defining search criteria.

    Parameters:
        root_folder (str): The directory containing multiple field_name subfolders.
        search_config (dict): Dictionary where keys are labels (e.g., "NDVI") 
                              and values are tuples (suffix, scale_factor).

    Returns:
        dict: {key_word: [list of matching file paths]}
    """
    matching_files = {key: [] for key in search_config}

    for field_name in os.listdir(root_folder):
        field_path = os.path.join(root_folder, field_name)

        if os.path.isdir(field_path):
            ms_folder = os.path.join(field_path, "MS WORK STATION")
            
            if os.path.isdir(ms_folder):
                for flight_folder in os.listdir(ms_folder):
                    flight_path = os.path.join(ms_folder, flight_folder)

                    if os.path.isdir(flight_path):
                        orthomosaic_folder = os.path.join(flight_path, "2_Orthomosaics")

                        if os.path.isdir(orthomosaic_folder):
                            for file in os.listdir(orthomosaic_folder):
                                for key_word, (suffix, _) in search_config.items():
                                    if file.endswith(suffix):
                                        matching_files[key_word].append(os.path.join(orthomosaic_folder, file))

    return matching_files

In [4]:
def rotate_image(image_array, angle):
    """Rotate an image by a given angle."""
    return rotate(image_array, angle, resize=True, mode='edge')

def preview_rotation(file_path, angle):
    """
    Rotates the first image and displays a preview.
    Returns the rotated image array for saving.
    """
    with rasterio.open(file_path) as src:
        image_array = src.read(1).astype(np.float32)
        rotated_array = rotate_image(image_array, angle)

    # Display the preview
    plt.figure(figsize=(6, 6))
    plt.imshow(rotated_array, cmap="gray")
    plt.title(f"Preview: Rotated {angle}°")
    plt.axis("off")
    plt.show()

    return rotated_array

In [5]:
def downscale_image(file_path, output_folder, scale_factor):
    """
    Downscale a TIFF/JPG file and save it in the structured output directory.

    Parameters:
        file_path (str): The path to the original file.
        output_folder (str): The base output folder.
        scale_factor (float): Scaling factor (e.g., 0.25 for 25%).

    Returns:
        str: Path to the saved downscaled file.
    """
    print(f"\n⚙️ Processing: {os.path.basename(file_path)} at {int(scale_factor * 100)}% resolution")

    try:
        with rasterio.open(file_path) as src:
            # Compute new dimensions
            new_width = int(src.width * scale_factor)
            new_height = int(src.height * scale_factor)
            print(f"   - Original size: {src.width} x {src.height}")
            print(f"   - New size: {new_width} x {new_height}")

            # Read and resample image
            resampled_array = src.read(
                out_shape=(src.count, new_height, new_width),
                resampling=Resampling.bilinear
            )

            # Update metadata
            profile = src.profile
            profile.update(
                width=new_width,
                height=new_height,
                transform=src.transform * src.transform.scale(
                    (src.width / new_width), (src.height / new_height)
                )
            )

            # Extract field_name from file path
            field_name = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(file_path)))))
            
            # Identify file type (TIFF/JPG) based on extension
            file_ext = os.path.splitext(file_path)[-1].lower()

            # Determine key_word (e.g., "NDVI", "RGB") based on the dictionary
            key_word = next((k for k, (suffix, _) in search_config.items() if file_path.endswith(suffix)), "Unknown")

            # Create structured output directory
            final_output_folder = os.path.join(output_folder, field_name, key_word)
            os.makedirs(final_output_folder, exist_ok=True)

            # Generate output filename
            output_scaled_path = os.path.join(
                final_output_folder,
                os.path.basename(file_path).replace(file_ext, f"_{int(scale_factor * 100)}%{file_ext}")
            )

            # Save downscaled image
            print(f"   - Saving downscaled file: {output_scaled_path}")
            with rasterio.open(output_scaled_path, "w", **profile) as dst:
                dst.write(resampled_array)

            print(f"✅ Successfully downscaled: {os.path.basename(file_path)}")
            return output_scaled_path

    except Exception as e:
        print(f"❌ ERROR processing {file_path}: {str(e)}")
        return None

In [8]:
# Example Usage
root_folder = r"D:\PhenoCrop\2_pix4d"  # Change to your actual root folder
output_folder = r"D:\PhenoCrop\2_2_Orthomosaics_resized"  # Base output directory

# Define search criteria with scale factors
search_config = {
    "RGB": ("_transparent_mosaic_group1.tif", 0.25), # JPG files scaled to 25%
    "NDVI": ("_index_ndvi.tif", 0.5), # TIFF files scaled to 50%
#     "Green": ("_index_green_green.tif", 0.5),
#     "NIR": ("_index_nir_nir.tif", 0.5),
#     "RedEdge": ("_index_red_edge_red_edge.tif", 0.5),
#     "Red": ("_index_red_red.tif", 0.5)
}

# Step 1: Find orthomosaic files
orthomosaic_files = find_orthomosaic_files(root_folder, search_config)
orthomosaic_files

{'RGB': ['D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STATION\\20240603 OAT_FRONTIERS M3M 20m MS 80 85\\2_Orthomosaics\\20240603 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif',
  'D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STATION\\20240603 OAT_FRONTIERS M3M 30m MS 80 85\\2_Orthomosaics\\20240603 OAT_FRONTIERS M3M 30m MS 80 85_transparent_mosaic_group1.tif',
  'D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STATION\\20240607 OAT_FRONTIERS M3M 20m MS 80 85\\2_Orthomosaics\\20240607 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif',
  'D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STATION\\20240607 OAT_FRONTIERS M3M 30m MS 80 85\\2_Orthomosaics\\20240607 OAT_FRONTIERS M3M 30m MS 80 85_transparent_mosaic_group1.tif',
  'D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STATION\\20240611 OAT_FRONTIERS M3M 20m MS 80 85\\2_Orthomosaics\\20240611 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif',
  'D:\\PhenoCrop\\2_pix4d\\OAT_FRONTIERS\\MS WORK STA

In [9]:
# Step 2: Process each file with corresponding scale factor
for key_word, file_list in orthomosaic_files.items():
    print(f"\n🔍 Processing {key_word} files...")
    for file_path in file_list:
        suffix, scale_factor = search_config[key_word]
        downscale_image(file_path, output_folder, scale_factor)


🔍 Processing RGB files...

⚙️ Processing: 20240603 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif at 25% resolution
   - Original size: 10095 x 11182
   - New size: 2523 x 2795
   - Saving downscaled file: D:\PhenoCrop\2_2_Orthomosaics_resized\OAT_FRONTIERS\RGB\20240603 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1_25%.tif
✅ Successfully downscaled: 20240603 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif

⚙️ Processing: 20240603 OAT_FRONTIERS M3M 30m MS 80 85_transparent_mosaic_group1.tif at 25% resolution
   - Original size: 7084 x 7449
   - New size: 1771 x 1862
   - Saving downscaled file: D:\PhenoCrop\2_2_Orthomosaics_resized\OAT_FRONTIERS\RGB\20240603 OAT_FRONTIERS M3M 30m MS 80 85_transparent_mosaic_group1_25%.tif
✅ Successfully downscaled: 20240603 OAT_FRONTIERS M3M 30m MS 80 85_transparent_mosaic_group1.tif

⚙️ Processing: 20240607 OAT_FRONTIERS M3M 20m MS 80 85_transparent_mosaic_group1.tif at 25% resolution
   - Original size: 10271 x 1

# Rotate Ortho at a certain angle

In [1]:
# User-defined rotation angles per project (if available)
rotation_dict = {
     'PHENO_CROP': -13.5,
     'OAT_FRONTIERS': -13.5,
     'DIVERSITY_OATS': -13.5,
     'PRO_BAR_VOLL': -29.5,
     'E166': -115.5,
     'PRO_BAR_SØRÅS': -68.5
}